In [1]:
%cd ..

/home/ai_n_zag@lab.graphicon.ru/temp/conv-cp-decomposition


In [2]:
import os
import time
from typing import Union
from tqdm import tqdm
from collections import deque
from statistics import mean
from functools import partial

import numpy as np
from matplotlib import pyplot as plt

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision.models import alexnet, AlexNet_Weights

from conv_cp.conv_cp import decompose_model
from conv_cp.imagenet.dataset import ImageNet

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
def acc_fn_1(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.argmax(dim=1)
    correct = (y_pred == y_true).sum().item()
    return correct / y_true.size(0)


def acc_fn_5(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.topk(5, dim=1).indices
    correct = (y_pred == y_true.unsqueeze(1)).sum().item()
    return correct / y_true.size(0)


def loss_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    verbose: bool = False,
    num_steps: int = 50,
) -> float:
    model.eval()
    model.to(device)

    total_loss = 0
    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Validation")
    for step in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            break
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred = model(x)
            loss = F.cross_entropy(y_pred, y)
            total_loss += loss.item()

        if verbose:
            loop.set_postfix(loss=total_loss / (step + 1))

    model.cpu()
    return total_loss / num_steps


def train_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    lr: float,
    num_steps: int = 1000,
    metric_len: int = 20,
    loss_tol: float = 1e-3,
    acc_tol: float = 0.95,
    verbose: bool = False,
):
    model.train()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    running_loss = deque(maxlen=metric_len)
    running_acc = deque(maxlen=metric_len)
    runngin_acc_5 = deque(maxlen=metric_len)

    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Training")
    for _ in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            data_iter = iter(dataloader)
            x, y = next(data_iter)

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
        running_acc.append(acc_fn_1(y_pred, y))
        runngin_acc_5.append(acc_fn_5(y_pred, y))

        if verbose:
            loop.set_postfix(
                loss=mean(running_loss),
                acc_1=mean(running_acc),
                acc_5=mean(runngin_acc_5),
            )

        if mean(running_loss) < loss_tol or mean(running_acc) > acc_tol:
            break

    optimizer.zero_grad()
    model.cpu()
    model.eval()

In [4]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
transform = AlexNet_Weights.IMAGENET1K_V1.transforms()

dataset = ImageNet(root_dir="data/val-images", transform=transform)
train_dataset, val_dataset = dataset.split(0.9)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [5]:
loss_fn = partial(
    loss_fn,
    dataloader=val_loader,
    device="cuda",
    num_steps=50,
    verbose=True,
)
train_fn = partial(
    train_fn,
    dataloader=train_loader,
    device="cuda",
    lr=1e-7,
    metric_len=10,
    num_steps=100,
    verbose=True,
)

In [6]:
predefined_ranks = {
    "features.0": 69,
    "features.3": 154,
    "features.6": 153,
    "features.8": 178,
    "features.10": 196,
    "classifier.1": 365,
    "classifier.4": 275,
    "classifier.6": 260,
}

cp_model = decompose_model(
    model,
    conv_rank=750,
    fc_rank=900,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.0,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
    predefined_ranks=predefined_ranks,
)

Computing losses for conv layers
Using predefined rank 69 for features.0
Using predefined rank 154 for features.3
Using predefined rank 153 for features.6
Using predefined rank 178 for features.8
Using predefined rank 196 for features.10
Computing losses for fc layers
Using predefined rank 365 for classifier.1
Using predefined rank 275 for classifier.4
Using predefined rank 260 for classifier.6
Initializing CP model
Decomposing features.0 with rank 69
Training model with features.0 decomposed


Training: 100%|██████████| 100/100 [00:38<00:00,  2.60it/s, acc_1=0.502, acc_5=0.764, loss=2.21]


Decomposing features.3 with rank 154
Training model with features.3 decomposed


Training: 100%|██████████| 100/100 [00:39<00:00,  2.52it/s, acc_1=0.52, acc_5=0.747, loss=2.18]


Decomposing features.6 with rank 153
Training model with features.6 decomposed


...

Decomposing classifier.1 with rank 365
Training model with classifier.1 decomposed


Training: 100%|██████████| 100/100 [00:38<00:00,  2.57it/s, acc_1=0.44, acc_5=0.69, loss=2.53] 


Decomposing classifier.4 with rank 275
Training model with classifier.4 decomposed


Training: 100%|██████████| 100/100 [00:38<00:00,  2.61it/s, acc_1=0.446, acc_5=0.709, loss=2.47]


Decomposing classifier.6 with rank 260


In [7]:
def evaluate(
    model: nn.Module, dataloader: DataLoader, device: Union[str, torch.device]
):
    model.eval()
    model.to(device)
    accs = []
    accs_5 = []
    times = []
    loop = tqdm(dataloader, desc="Evaluation")
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            t_start = time.time()
            y_pred = model(x)
            times.append(time.time() - t_start)
            accs.append(acc_fn_1(y_pred, y))
            accs_5.append(acc_fn_5(y_pred, y))
            loop.set_postfix(acc=mean(accs), acc_5=mean(accs_5))
    return mean(accs), mean(accs_5), sum(times)

In [8]:
acc_1, acc_5, inference_time = evaluate(cp_model, val_loader, "cuda")
print(f"Top-1 Accuracy: {acc_1} | Top-5 Accuracy: {acc_5} | Inference time: {inference_time}")

Evaluation: 100%|██████████| 20/20 [00:08<00:00,  2.32it/s, acc=0.405, acc_5=0.694]

Top-1 Accuracy: 0.4049402573529412 | Top-5 Accuracy: 0.6939682904411765 | Inference time: 0.058960914611816406


In [9]:
orig_model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
cp_model.cpu()
orig_model.cuda()
ref_acc_1, ref_acc_5, ref_inference_time = evaluate(orig_model, val_loader, "cuda")
print(f"Top-1 Accuracy: {ref_acc_1} | Top-5 Accuracy: {ref_acc_5} | Inference time: {ref_inference_time}")

Evaluation: 100%|██████████| 20/20 [00:08<00:00,  2.38it/s, acc=0.566, acc_5=0.791]

Top-1 Accuracy: 0.5660960477941176 | Top-5 Accuracy: 0.7906135110294118 | Inference time: 0.4947676658630371


In [10]:
def get_size_ratio(model1: nn.Module, model2: nn.Module) -> float:
    size1 = sum(p.numel() for p in model1.parameters())
    size2 = sum(p.numel() for p in model2.parameters())
    return size1 / size2

def get_model_size(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters())

size_ratio = get_size_ratio(cp_model, orig_model)
cp_size = get_model_size(cp_model)
orig_size = get_model_size(orig_model)
print(f"Orig model size: {orig_size} | CP model size: {cp_size} | Size Ratio: {size_ratio:.3f}" )

Orig model size: 61100840 | CP model size: 8799651 | Size Ratio: 0.144
